In [1]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 3.4 MB/s  0:00:07m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 2.6 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 2.5 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 861.5/861.5 kB 2.4 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18/18 [gradio]17/18 [gradio]client]b]


In [2]:
!pip install -q --upgrade openai python-dotenv

In [4]:
import os
from IPython.display import display, Markdown
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")              # set in .env or env
openai_base_url = os.getenv("OPENAI_BASE_URL", "http://127.0.0.1:1234")

openai_client = OpenAI(api_key=openai_api_key, base_url=openai_base_url)

print("OpenAI Client initialized successfully.")
print("Base URL:", openai_base_url)

OpenAI Client initialized successfully.
Base URL: http://127.0.0.1:1234/v1


In [5]:
def print_markdown(text):
    """Display text as Markdown in Jupyter."""
    display(Markdown(text))

Building a basic AI tutor function (no gradio yet)

core function:

1. Take a user's question as input.
2. Construct a prompt for the OpenAI API, telling it to act as a helpful tutor.
3. Call the OpenAI API
4. return the AI's answer.

In [16]:
def get_ai_tutor_response(user_question):
    """
    Sends a question to the OpenAI API, asking it to respond as an AI tutor.

    Args:
        user_question(str): The question asked by the user.

    returns:
        str: The AI's response, or an error message.

    """
    system_promt = "You are a helpful and patient AI tutor.Explain concepts clearly and provide examples when necessary."

    try:
        response = openai_client.chat.completions.create(
           model="gemma-3-1b",
           messages=[
                {"role": "system", "content": system_promt},
                {"role": "user", "content": user_question}
                ],
        max_tokens=512,
        temperature=0.7,
)

        ai_response = response.choices[0].message.content
        return ai_response
    
    except Exception as e:
        return f"An error occurred: {str(e)}"

In [17]:
test_question = "Could you explain the concept of functions in python and their purpose in programming?"
print_markdown(f"Asking the AI tutor: {test_question}")

tutor_answer = get_ai_tutor_response(test_question)

print_markdown(f"AI Tutor's Response:\n")
print_markdown(tutor_answer)

Asking the AI tutor: Could you explain the concept of functions in python and their purpose in programming?

AI Tutor's Response:


Okay, let's dive into the concept of functions in Python! It’s a fundamental part of programming, and understanding them is key to writing clean and efficient code.

**What is a Function in Python?**

At its simplest, a function in Python is a block of code that performs a specific task.  Think of it like a mini-program within your larger program. You give the function a name, and you provide it with input (arguments) – data that’s needed for the function to work.  The function then does something with that input, and it returns a result (which is usually another value or a new value depending on the function's purpose).

**Let’s break it down with an example:**

```python
def greet(name):  # This is the function definition. 
    """This function greets a person by name."""
    print(f"Hello, {name}!")  # 'f-string' - a modern way to embed variables in strings.
    return "Hello!"  # The function returns a value (in this case, the string "Hello!")

# Calling the function
person = greet("Alice")  # Pass the argument "Alice" to the function.
print(person) # Printing the returned value (which is "Hello!")
```

**Explanation:**

*   `def greet(name):`  This line *defines* a function named `greet`. The parentheses `()` indicate that it's a function definition.  The name of the function is what you give it – `greet`.
*   `"""This function greets a person by name."""`:  This is a *docstring*. It's like a little note explaining what the function does.  It’s good practice to include docstrings for your code.
*   `print(f"Hello, {name}!")`: This line is the *body* of the function.  It's where the action happens – it displays a greeting message.  The `f` before the string means we're using an f-string, which is a very convenient way to embed variables directly into the string.
*   `return "Hello!"`:  This line *returns* a value from the function.  The `return` keyword is essential. The function will *produce* a value and pass it back to the part of the program that called the function.
*   `person = greet

build an interactive interface using gradio

In [18]:
import gradio as gr

In [25]:
ai_tutor_interface_simple = gr.Interface(
    fn=get_ai_tutor_response,
    inputs=gr.Textbox(lines=4, label="Ask the AI Tutor a Question"),
    outputs=gr.Markdown(label="AI Tutor's Response"),
    title="AI Tutor",
    description="Ask programming-related questions and get explanations from the AI tutor.",    
    flagging_mode="never",
)

print("Launching AI Tutor interface...")
ai_tutor_interface_simple.launch()

Launching AI Tutor interface...
* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Add Streaming

In [30]:
def stream_ai_tutor_response(user_question):
    """
    Sends a question to the OpenAI API, asking it to respond as an AI tutor, and streams the response.

    Args:
        user_question(str): The question asked by the user.
        
    yields:
        str: The AI's response in chunks, or an error message.

    """

    system_prompt = "You are a helpful and patient AI tutor. Explain concepts clearly and provide examples when necessary."

    try:

        stream = openai_client.chat.completions.create(
            model = "gemma-3-1b",
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_question}
            ],
            max_tokens = 512,
            temperature = 0.7,
            stream = True,
        )

        full_response = "" #keep track of full response if needed later

        for chunk in stream:
            if chunk.choices[0].delta and chunk.choices[0].delta.content:
                text_chunk = chunk.choices[0].delta.content
                full_response += text_chunk
                yield full_response

    except Exception as e:
        print(f"An error occurred: {str(e)}")


In [45]:
def stream_ai_tutor_response_with_level(user_question, explanation_level_value):
    """
    Sends a question to the OpenAI API, asking it to respond as an AI tutor with a specified explanation level, and streams the response.

    Args:
        user_question (str): The question asked by the user.
        explanation_level_value (int): The desired explanation level (1-5).

    Yields:
        str: Chunks of the AI's response (progressive).
    """
    explanation_levels = {
        1: "like I'm five years old",
        2: "like I'm 10 years old",
        3: "like a high school student",
        4: "like a college student",
        5: "like an expert in the fields",
    }

    level_description = explanation_levels.get(explanation_level_value, "clearly and concisely")
    system_prompt = f"You are a helpful and patient AI tutor. Explain concepts {level_description}."

    try:
        stream = openai_client.chat.completions.create(
            model="gemma-3-1b",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_question}
            ],
            max_tokens=512,
            temperature=0.7,
            stream=True,
        )

        full_response = ""
        for chunk in stream:
            # many local clients stream chunks with chunk.choices[0].delta.content
            try:
                delta = chunk.choices[0].delta
                content = getattr(delta, "content", None) if delta is not None else None
            except Exception:
                content = None

            if content:
                full_response += content
                yield full_response

    except Exception as e:
        yield f"An error occurred: {str(e)}"

In [49]:
ai_tutor_interface_simple = gr.Interface(fn = stream_ai_tutor_response_with_level,
    inputs =[
        gr.Textbox(lines=4, label="Ask the AI Tutor a Question"),
        gr.Slider(minimum=1, maximum=5, step=1, 
                  label="Explanation Level")
    ],
    outputs=gr.Markdown(label="AI Tutor's Response(Streaming)", container=True, height = 250),
             title= "Advanced AI Tutor",
             description="Ask programming-related questions and get explanations from the AI tutor.",
             flagging_mode="never",    
    )

print("Launching Advanced AI Tutor interface...")
ai_tutor_interface_simple.launch()

Launching Advanced AI Tutor interface...
* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
